## Chapter 8 Spatial Models

In [ ]:
rm(list = ls())
# install.packages("brms")
# install.packages("tidyverse")
# install.packages("modelr")
# install.packages("tidybayes")
library(brms)
library(modelr)
library(tidybayes)
library(dplyr)
#library(cowplot)
source("/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/configurations/configurations.r")
source("/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/configurations/user_functions.r")
data_location = "/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/clean_data/"
setwd("/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/Chapters/Chapters/Chapter8")
# for all chapters
##### Data Recodes #####
load("/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/clean_data/pooled.auth.rda")  ### Just work from this data; everything should be here, recoded.
data$authoritarianism<-(rowMeans(cbind(data$auth.1.x, data$auth.2.x,data$auth.3.x, data$auth.4.x), na.rm=T)-1)
data$party3<-car::recode(data$pid*6+1, "1:2='Democrat'; 3:5='Independent'; 6:7='Republican'; else=NA" )
data$republican<-car::recode(data$pid*6+1, "1:2=0; 3:5=0; 6:7=1" )
data$democrat<-car::recode(data$pid*6+1, "1:2=1; 3:5=0; 6:7=0" )
data$independent<-car::recode(data$pid*6+1, "1:2=0; 3:5=1; 6:7=0" )
cor.test(data$know.interview.pre, data$know.interview.post)
data$knowledge<-(rowMeans(cbind(data$know.interview.pre, data$know.interview.post), na.rm=T)-1)/4

data$ideologyCD<-(data$ideologyCD-1)/6
data$ideologyCR<-(data$ideologyCR-1)/6

data$ideologyD<-(data$ideologyD-1)/6
data$ideologyR<-(data$ideologyR-1)/6

### Difference measures
  data$difference_2<-(data$ideologyR-data$ideologyD)^2
  data$difference_abs<-data$ideologyR-data$ideologyD %>% abs()
  data$difference<-data$ideologyR-data$ideologyD 

  data$differenceR1<-data$ideology-data$ideologyR
  data$differenceD1<-data$ideology-data$ideologyD
  data$proximity<-(data$ideology-data$ideologyD)^2-(data$ideology-data$ideologyR)^2
  data$differenceR2<-(data$ideology-data$ideologyR)^2
  data$differenceD2<-(data$ideology-data$ideologyD)^2  
  data$mode<-as.character(data$mode)
  


  data$diffR<-data$ideology-data$ideologyR
  data$diffD<-data$ideology-data$ideologyD
  
  data$diffCR<-data$ideology-data$ideologyCR
  data$diffCD<-data$ideology-data$ideologyCD

data = data %>%
  mutate(cenD = if_else(diffD >=1, "right", "none")) %>%
  mutate(cenR = if_else(diffR <=-1, "left", "none"))

ggtheme = theme(
        plot.title=element_text(face="bold",hjust=0,vjust=0,colour="#3C3C3C",size=20),
        axis.text.x=element_text(size=16,colour="#535353",face="bold"),
        axis.text.y=element_text(size=16,colour="#535353",face="bold"),
        axis.title = element_text(size=16,colour="#535353",face="bold"),
        axis.title.y=element_text(size=16,colour="#535353",face="bold",vjust=1.5),
        axis.ticks=element_blank(),
        strip.text.x = element_text(size = 16),
        panel.grid.major=element_line(colour="#D0D0D0",size=.25),
        panel.background=element_rect(fill="white"),
        legend.text=element_text(size=14),
        legend.title=element_text(size=16)) 

We may choose not to present all this, but it might still be useful in constructing our narrative. The place to start with a spatial model is to first examine trends over time, particularly since the ideology of the respondent, and perceptions may change over time. 

## Perceptions of the Parties

Since the items are all measured on the same metric, we can use the perceptions of the parties as the candidate's ideal point.



In [ ]:
### I create smaller, more tractable versions of the data rather than operating on the full data frame, data.
tmp_dat = data[,c("ideologyCR", "ideologyCD", "vote", "ideology", "authoritarianism", 
                 "female", "age", "college", "income",
                 "jewish", "catholic", "other", "year")] %>% na.omit() %>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism) 

bf1 = bf(mvbind(ideologyCR, ideologyCD) ~ authoritarianism + authoritarianism_2 + 
                 (1+authoritarianism + authoritarianism_2|year))
     
model_ideology1 <- brm(bf1 + set_rescor(FALSE), 
                 family = "gaussian",
                  data = tmp_dat,
                  chains = 3, 
                  cores = 8, 
                  seed = 1234, 
                  iter = 3000)             

## Alignment

This is part of the story, but really we're interested in one's perceived distance -- how far is the candidate from one's ideological ideal point?

In [ ]:
### I create smaller, more tractable versions of the data rather than operating on the full data frame, data.
tmp_dat = data[,c("diffCR", "diffCD", "vote", "ideology", "authoritarianism", 
                 "female", "age", "college", "income",
                 "jewish", "catholic", "other", "year")] %>% na.omit() %>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism) 

bf1 = bf(mvbind(diffCR, diffCD) ~ authoritarianism + authoritarianism_2 + 
                 (1+authoritarianism + authoritarianism_2|year))
     
model_ideology2 <- brm(bf1 + set_rescor(FALSE), 
                 family = "gaussian",
                  data = tmp_dat,
                  chains = 3, 
                  cores = 8, 
                  seed = 1234, 
                  iter = 3000)             

# Building a Spatial Model

The first thing to consider is that some voters incorrectly locate the parties. I'll create several variables

* Ideological knowledge. Create a dummy coded 1 if participants correctly locate the Republican candidate to the right of the Democratic candidate, 0 if the answer was incorrect.

* Indifference. I'll address this in a slightly different way in a bit, but we could also classify those with a score of zero on both scales to be completely indifferent to one candidate or the other.

* Extremists. These are individuals who rate themselves to the "left" of the Democratic candidate and individuals who rate themselves to the right of the Republican candidate.

In [ ]:
tmp_dat = data[,c("ideologyCR", "ideologyCD", "diffCD", "diffCR", "authoritarianism", 
                 "female", "age", "college", "income",
                 "jewish", "catholic", "other", "year")] %>% na.omit() %>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism) %>% 
                mutate(confused    = ifelse(ideologyCR < ideologyCD, 1, 0)) %>% 
                mutate(indifferent = ifelse(diffCR ==  diffCD, 1, 0)) %>% 
                mutate(extremeR     = ifelse((diffCD > 0 & diffCR > 0), 1, 0))  %>%
                mutate(extremeD     = ifelse((diffCD < 0 & diffCR < 0), 1, 0))  
fit1 <- brm(confused~  authoritarianism + authoritarianism_2 + 
                 (1+authoritarianism + authoritarianism_2|year), 
                 family = bernoulli(link = "probit"),
                 data = tmp_dat, 
                 chains = 3, cores = 8, seed = 1234, 
                 iter = 2000)
fit2 <- brm(indifferent~  authoritarianism + authoritarianism_2 + 
                 (1+authoritarianism + authoritarianism_2|year), 
                 family = bernoulli(link = "probit"),
                 data = tmp_dat, 
                 chains = 3, cores = 8, seed = 1234, 
                 iter = 2000)
fit3 <- brm(extremeR~  authoritarianism + authoritarianism_2 + 
                 (1+authoritarianism + authoritarianism_2|year), 
                 family = bernoulli(link = "probit"),
                 data = tmp_dat, 
                 chains = 3, cores = 8, seed = 1234, 
                 iter = 2000)
fit4 <- brm(extremeD~  authoritarianism + authoritarianism_2 + 
                 (1+authoritarianism + authoritarianism_2|year), 
                 family = bernoulli(link = "probit"),
                 data = tmp_dat, 
                 chains = 3, cores = 8, seed = 1234, 
                 iter = 2000)

# Do Authoritarians "Vote Correctly"?

To build the spatial model, it's important to rescale the ideology variable. The derivation is below.

In [ ]:
tmp_dat = data[,c("vote", "ideology", "authoritarianism", "ideologyCR", "ideologyCD",
                 "female", "age", "college", "income", "party3",
                 "jewish", "catholic", "other", "year")] %>% na.omit() %>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism) %>% 
                 mutate(confused    = ifelse(ideologyCR <= ideologyCD, 1, 0)) %>% 
                 filter(confused == 0) %>% 
                 group_by(year) %>% 
                 mutate(mean_democrat  = mean(ideologyCD)) %>% 
                 mutate(mean_republican= mean(ideologyCR)) %>% 
                 mutate(ideology_converted = (0.5 * (ideology - mean_democrat)/(mean_republican - mean_democrat)-0.25)) %>%               
                 mutate(republican = ifelse(party3  == "Republican", 1, 0))%>% 
                 mutate(independent = ifelse(party3 == "Independent", 1, 0))%>% 
                 mutate(democrat = ifelse(party3 == "Democrat", 1, 0))%>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism)

fit_spatial1 <- brm(vote~ ideology_converted +  (1 +ideology_converted|year), 
                 family = bernoulli(link = "probit"),
                 data = tmp_dat,
                 chains = 2, 
                 cores = 6, 
                 seed = 1234, 
                iter = 4000,
                control = list(max_treedepth = 15))   

fit_spatial2 <- brm(vote~ republican + independent + ideology_converted +  (1 + republican + independent +
                            ideology_converted |year), 
                 family = bernoulli(link = "probit"),
                 data = tmp_dat,
                 chains = 2, 
                 cores = 6, 
                 seed = 1234, 
                iter = 10000,
                control = list(max_treedepth = 15)) 
fit_spatial3 <- brm(vote~  authoritarianism + authoritarianism_2 + ideology_converted + 
                           authoritarianism:ideology_converted + authoritarianism_2:ideology_converted + 
                 (1 +  authoritarianism:ideology_converted + authoritarianism_2:ideology_converted +
                 authoritarianism + authoritarianism_2 + ideology_converted|year),
                 family = bernoulli(link = "probit"),
                 data = tmp_dat,
                 chains = 2, 
                 cores = 6, 
                 seed = 1234, 
                 iter = 10000,
                 control = list(max_treedepth = 15)) 


# Education Effects

In [ ]:
tmp_dat = data[,c("vote", "ideology", "authoritarianism", "ideologyCR", "ideologyCD",
                 "female", "age", "college", "income", "party3",
                 "jewish", "catholic", "other", "year", "college")] %>% na.omit() %>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism) %>% 
                 mutate(confused    = ifelse(ideologyCR <= ideologyCD, 1, 0)) %>% 
                 filter(confused == 0) %>% 
                 group_by(year) %>% 
                 mutate(mean_democrat  = mean(ideologyCD)) %>% 
                 mutate(mean_republican= mean(ideologyCR)) %>% 
                 mutate(ideology_converted = (0.5 * (ideology - mean_democrat)/(mean_republican - mean_democrat)-0.25)) %>%               
                 mutate(republican = ifelse(party3  == "Republican", 1, 0))%>% 
                 mutate(independent = ifelse(party3 == "Independent", 1, 0))%>% 
                 mutate(democrat = ifelse(party3 == "Democrat", 1, 0))%>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism)

fit_spatial4 <- brm(vote~  authoritarianism + authoritarianism_2 + ideology_converted + college + 
                           college:ideology_converted + college:ideology_converted +
                           college:authoritarianism + college:authoritarianism_2 +
                           authoritarianism:ideology_converted + authoritarianism_2:ideology_converted + 
                           authoritarianism:ideology_converted:college + authoritarianism_2:ideology_converted:college +
                 (1 +  authoritarianism + authoritarianism_2 + ideology_converted + college + 
                           college:ideology_converted + college:ideology_converted +
                           college:authoritarianism + college:authoritarianism_2 +
                           authoritarianism:ideology_converted + authoritarianism_2:ideology_converted + 
                           authoritarianism:ideology_converted:college + authoritarianism_2:ideology_converted:college|year),
                 family = bernoulli(link = "probit"),
                 data = tmp_dat,
                 chains = 2, 
                 cores = 6, 
                 seed = 1234, 
                 iter = 10000,
                 control = list(max_treedepth = 15)) 


In [22]:
# Save data
spatial = list(model_ideology1, model_ideology2, fit1, fit2, 
               fit3, fit4, fit_spatial1, fit_spatial2, fit_spatial3, fit_spatial4)
save(spatial, file = "spatial.rda")

# Perceived Partisan Polarization 

Are authoritarians motivated by the perception that the political parties are divided?

In [ ]:
tmp_dat = data[,c("vote", "ideology", "authoritarianism", "ideologyCR", "ideologyCD",
                 "female", "age", "college", "income", "party3",
                 "jewish", "catholic", "other", "year", "difference", "difference_2", "difference_abs")] %>% na.omit() %>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism) %>% 
                 mutate(confused    = ifelse(ideologyCR <= ideologyCD, 1, 0)) %>% 
                 filter(confused == 0) %>% 
                 group_by(year) %>% 
                 mutate(mean_democrat  = mean(ideologyCD)) %>% 
                 mutate(mean_republican= mean(ideologyCR)) %>% 
                 mutate(ideology_converted = (0.5 * (ideology - mean_democrat)/(mean_republican - mean_democrat)-0.25)) %>%               
                 mutate(republican = ifelse(party3  == "Republican", 1, 0))%>% 
                 mutate(independent = ifelse(party3 == "Independent", 1, 0))%>% 
                 mutate(democrat = ifelse(party3 == "Democrat", 1, 0))%>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism)%>% 
                 mutate(difference = (difference)^2)

difference_model <- brm(vote~ 
                          authoritarianism + authoritarianism_2 + ideology_converted + difference_2 + 
                          difference_2:authoritarianism + difference_2:authoritarianism_2 + ideology_converted:difference_2 + 
                          difference_2:ideology_converted:authoritarianism + difference_2:ideology_converted:authoritarianism_2 +
                                    (1 + authoritarianism + authoritarianism_2 + 
                                         ideology_converted + difference_2 + 
                                         difference_2:authoritarianism + difference_2:authoritarianism_2 + 
                                         ideology_converted:difference_2 + 
                                         difference_2:ideology_converted:authoritarianism + 
                                         difference_2:ideology_converted:authoritarianism_2|year), 
                 family = bernoulli(link = "probit"),
                 data = tmp_dat,
                 chains = 2, 
                 cores = 8, 
                 seed = 1234, 
                iter = 5000,
                control = list(max_treedepth = 15)) 


# difference_model1 <- brm(vote~ 
#                           authoritarianism + authoritarianism_2 + ideology_converted + difference + 
#                           difference:authoritarianism + difference:authoritarianism_2 + ideology_converted:difference + 
#                           difference:ideology_converted:authoritarianism + difference:ideology_converted:authoritarianism_2 +
#                                     (1 + authoritarianism + authoritarianism_2 + 
#                                          ideology_converted + difference + 
#                                          difference:authoritarianism + difference:authoritarianism_2 + 
#                                          ideology_converted:difference + 
#                                          difference:ideology_converted:authoritarianism + 
#                                          difference:ideology_converted:authoritarianism_2|year), 
#                  family = bernoulli(link = "probit"),
#                  data = tmp_dat,
#                  chains = 2, 
#                  cores = 8, 
#                  seed = 1234, 
#                 iter = 5000,
#                 control = list(max_treedepth = 15)) 




In [7]:
save(difference_model, file = "diff_model.rda")


------

\begin{equation}
x' &=& &x \sin\phi &+& z \cos\phi \\
z' &=& - &x \cos\phi &+& z \sin\phi \\
\end{equation}